In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [2]:
path_to_data_file = ("data/DamageMatrixVariables.mat.xlsx") 
df = pd.read_excel(path_to_data_file, "bldgs_inputs_all") #pip3 install openpyxl
df.drop('Extra', 1, inplace = True) #dropping the Extra Column
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '_')\
                .str.replace('%', 'percent').str.replace(':','_').str.replace('ind_', 'ind').str.replace('__', '_')
df.columns

Index(['hazard', 'secondary_hazard', 'event_size', 'median_year_built',
       'occupancy', 'walls', 'roofing', 'roof_type', 'height', 'footprint',
       'percent_owner_occupied', 'percent_renter_occupied', 'roughness',
       'imp_surfaces', 'forested', 'housing_density', 'pop_density', 'pop',
       'median_age', 'percent_african_american', 'percent_native_american',
       'percent_asain', 'percent_hispanic', 'ext_ind', 'service_ind',
       'income_poverty', 'per_capita_income'],
      dtype='object')

In [3]:
for i in df.columns:
    print('the unique values in ' + str(i) ,  df[i].unique()) 

the unique values in hazard [120  90  96  87 103  93  89  79  84  95  75  80  65 140 130 101 104  74
  85 119 100 110  97 125 132 115 150  60 135  70 112 131  83 160 127  61
  94  98 134]
the unique values in secondary_hazard [31 32]
the unique values in event_size [3 1 2]
the unique values in median_year_built [1959 1982 1983 1988 1985 2000 1987 1978 1977 1981 1990 1965 1967 1971
 1976 1956 1939 1994 1999 1986 1989 2001 1984 1979 1975 1973 1998 1970
 1980 1974 1992]
the unique values in occupancy [303.4 310.5 311.2 309.1 312.1 303.3 304.1 310.4 311.3 305.1 310.2 306.2]
the unique values in walls [ 7.9   5.11  5.3   0.5   5.5   5.1   5.8   5.31  5.13  6.    5.    1.
  8.5   7.1   7.    6.3  10.9   6.1   5.32  6.5   8.  ]
the unique values in roofing [ 7.2  5.2  5.8  0.5  8.8 11.4  7.4  5.4 10.4  4.  11.8 11.5  5.5  8.
 10.2  5.1  5. ]
the unique values in roof_type [ 2.3   6.1  10.22  6.2   7.   10.23  1.   11.    1.3   1.2  11.2  10.24
 10.13 10.14 10.12  9.    4.1 ]
the unique values

In [4]:
categorical_columns = ['secondary_hazard', 'event_size',\
                       'height', 'footprint', 'roughness'] #included everything with 5 and below unique values.

df_category = df[categorical_columns]
df_numerical = df.drop(categorical_columns, 1)

In [5]:
df_category.isnull().sum()

secondary_hazard    0
event_size          0
height              0
footprint           1
roughness           2
dtype: int64

In [6]:
df_numerical.isnull().sum()

hazard                      0
median_year_built           0
occupancy                   0
walls                       0
roofing                     0
roof_type                   0
percent_owner_occupied      0
percent_renter_occupied     0
imp_surfaces                2
forested                    2
housing_density             0
pop_density                 0
pop                         0
median_age                  0
percent_african_american    0
percent_native_american     0
percent_asain               0
percent_hispanic            0
ext_ind                     0
service_ind                 0
income_poverty              0
per_capita_income           0
dtype: int64

In [7]:
df_category['footprint'].fillna(df_category.footprint.median(), inplace=True)
df_category['roughness'].fillna(df_category.roughness.median(), inplace=True)

df_numerical['imp_surfaces'].fillna(df_numerical.imp_surfaces.median(), inplace=True)
df_numerical['forested'].fillna(df_numerical.forested.median(), inplace=True)

## One_Hot_Encoding

In [8]:
onehotencoder = preprocessing.OneHotEncoder()
frames = []
for i in df_category.columns:
    X = onehotencoder.fit_transform(df_category[i].values.reshape(-1,1) ).toarray()
    data = pd.DataFrame(X, columns = [str(i)+'_'+str(int(a)) for a in np.sort(df_category[i].unique())] )
    frames.append(data)
df_category_onehot = pd.concat(frames, axis =1)

In [9]:
df_category_onehot.head(20)

,secondary_hazard_31,secondary_hazard_32,event_size_1,event_size_2,event_size_3,height_1,height_2,height_3,height_6,footprint_1,footprint_2,footprint_3,footprint_4,footprint_5,roughness_2,roughness_3
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


## Standard_Scalar (for numerical features) 

In [11]:
standardscaler = preprocessing.StandardScaler() 

data_numericals = standardscaler.fit_transform(df_numerical)
df_numerical_standard = pd.DataFrame(data_numericals, columns= [str(i) for i in df_numerical.columns])
df_numerical_standard.head(20)

,hazard,median_year_built,occupancy,walls,roofing,roof_type,percent_owner_occupied,percent_renter_occupied,imp_surfaces,forested,...,pop,median_age,percent_african_american,percent_native_american,percent_asain,percent_hispanic,ext_ind,service_ind,income_poverty,per_capita_income
0,0.932157,-1.518464,-2.890064,1.623151,0.822069,-1.314405,-1.939309,1.324811,2.598658,-1.838361,...,-1.409104,1.825205,5.803078,-0.445197,-0.515700,-0.663634,-1.251306,1.312868,-1.385465,-0.538326
1,-0.371897,0.067173,0.210913,0.015562,-0.178265,-0.061866,1.234033,-1.124597,1.164493,-0.290964,...,-0.796610,1.401525,1.252809,-0.445197,-0.515700,0.242886,-1.251306,-0.313488,-0.892160,0.475124
2,-0.111086,0.136114,0.210913,0.125040,0.121836,1.296149,-0.089272,-0.454819,-1.089195,2.030132,...,-0.322903,-0.843981,-0.446176,-0.445197,-0.515700,-0.663634,-0.272140,0.191066,-0.301567,-0.742376
3,-0.502302,0.136114,0.210913,-2.640704,-2.529049,-0.061866,-0.089272,-0.454819,-1.089195,1.643283,...,-0.322903,-0.843981,-0.446176,-0.445197,-0.515700,-0.663634,-0.272140,0.191066,-0.301567,-0.742376
4,0.193193,0.480818,0.210913,0.240279,-0.178265,-0.061866,0.323328,-0.322310,-1.089195,0.482735,...,-0.749584,0.328201,-0.231375,-0.445197,-0.515700,-0.538604,-0.331349,0.995013,-0.727343,-0.112121
5,-0.241491,0.480818,0.210913,0.009800,-0.178265,1.296149,0.323328,-0.322310,-1.089195,0.482735,...,-0.749584,0.328201,-0.231375,-0.445197,-0.515700,-0.538604,-0.331349,0.995013,-0.727343,-0.112121
6,-0.415365,0.273995,0.516643,0.413138,0.121836,-0.028905,-2.128260,2.807694,0.959612,0.095886,...,-0.987011,1.246175,1.895378,0.732163,-0.021675,0.123849,-0.931975,3.607038,-1.008905,-1.446810
7,-0.850050,1.308107,0.210913,0.130802,-0.178265,-0.028905,0.375426,0.087999,-0.269672,0.482735,...,0.229947,-1.154680,-0.446176,-0.445197,-0.515700,-0.460033,-0.476835,-0.004260,0.250111,-0.379731
8,-0.632708,1.308107,0.210913,0.125040,-0.178265,0.234788,0.375426,0.087999,-0.269672,0.482735,...,0.229947,-1.154680,-0.446176,-0.445197,-0.515700,-0.460033,-0.476835,-0.004260,0.250111,-0.379731
9,-0.850050,1.308107,0.210913,0.130802,-0.178265,-0.028905,0.375426,0.087999,-0.269672,0.482735,...,0.229947,-1.154680,-0.446176,-0.445197,-0.515700,-0.460033,-0.476835,-0.004260,0.250111,-0.379731


In [14]:
final = [df_numerical_standard, df_category_onehot]

final_df = pd.concat(final, axis = 1)

In [15]:
final_df

,hazard,median_year_built,occupancy,walls,roofing,roof_type,percent_owner_occupied,percent_renter_occupied,imp_surfaces,forested,...,height_2,height_3,height_6,footprint_1,footprint_2,footprint_3,footprint_4,footprint_5,roughness_2,roughness_3
0,0.932157,-1.518464,-2.890064,1.623151,0.822069,-1.314405,-1.939309,1.324811,2.598658,-1.838361,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,-0.371897,0.067173,0.210913,0.015562,-0.178265,-0.061866,1.234033,-1.124597,1.164493,-0.290964,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-0.111086,0.136114,0.210913,0.125040,0.121836,1.296149,-0.089272,-0.454819,-1.089195,2.030132,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,-0.502302,0.136114,0.210913,-2.640704,-2.529049,-0.061866,-0.089272,-0.454819,-1.089195,1.643283,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.193193,0.480818,0.210913,0.240279,-0.178265,-0.061866,0.323328,-0.322310,-1.089195,0.482735,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,-1.458609,-0.415412,0.909725,-0.047819,0.121836,-0.028905,-0.982653,1.053138,-1.089195,-0.677813,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
113,-1.067392,-0.415412,0.909725,0.413138,0.121836,-0.028905,-0.982653,1.053138,-1.089195,-0.677813,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
114,-1.023924,-0.415412,0.516643,0.413138,1.622336,-0.028905,-0.982653,1.053138,-1.294076,-0.677813,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
115,1.801527,-0.001768,0.210913,0.027086,-0.178265,-0.028905,-1.219978,0.321312,-1.089195,-1.451511,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [16]:
final_df.to_csv('data/preprocessed.csv', index = False) 